In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

In [2]:
df_train = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q1. Downloading the data.

Read the data for January. How many columns are there ?

In [3]:
df_train.shape

(3066766, 19)

**Answer**: 19

## Q2. Computing duration

What's the standard deviation of the trips duration in January ?

In [4]:
df_train["duration"] = df_train["tpep_dropoff_datetime"] - df_train["tpep_pickup_datetime"]
df_train["duration"] = df_train["duration"].apply(lambda td: td.total_seconds() / 60)

In [5]:
df_train["duration"].std()

42.59435124195458

**Answer**: 42.59

## Q3. Dropping outliers

What fraction of the records left after you dropped the outliers?

In [6]:
df_train = df_train[(df_train["duration"] >= 1) & (df_train["duration"] <= 60)]

In [7]:
df_train.shape

(3009173, 20)

In [8]:
3009173*100 / 3066766

98.1220282212598

**Answer**: 98%

## Q4. One-hot encoding

What's the dimensionality of this matrix (number of columns) ?

In [9]:
dv = DictVectorizer()
features = ["PULocationID", "DOLocationID"]
df_train[features] = df_train[features].fillna(-1).astype('int').astype('str')

train_dicts = df_train[features].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [10]:
X_train.shape

(3009173, 515)

**Answer**: 515

## Q5. Training a model

In [11]:
lr = LinearRegression()
y_train = df_train["duration"].values
lr.fit(X_train, y_train)

LinearRegression()

In [12]:
y_pred = lr.predict(X_train)

In [13]:
root_mean_squared_error(y_train, y_pred)

7.649261932106969

**Answer**: 7.64

In [14]:
del y_pred

## Q6. Evaluating the model

In [15]:
features = ["PULocationID", "DOLocationID"]

def read_data(filename):
    df = pd.read_parquet(filename)

    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60)

    df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

    df[features] = df[features].fillna(-1).astype('int').astype('str')

    return df

In [16]:
df_test = read_data("data/yellow_tripdata_2023-02.parquet")

: 

In [ ]:
test_dicts = df_test[features].to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [ ]:
y_test = df_test["duration"].values

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
root_mean_squared_error(y_test, y_pred)

7.811819793542861

**Answer**: 7.81